## Import necessary libraries and setup the initial key/url

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
import json
def get_keys(path):
    with open(path) as f:
        return json.load(f)

api_key = get_keys("/Users/takehiroyasuoka/Documents/Flatiron/Flatiron Repos/Taki's Playground/API_keys.json")
moviedb_key = list(api_key.values())[0]

In [3]:
moviedb_url ="https://api.themoviedb.org/3/discover/movie?api_key="+moviedb_key+"&language=en-US&sort_by=popularity.desc&include_adult=false&include_video=false&page=1&year=2019"
movie_url = requests.get(moviedb_url)

## Create a List of Urls with the Years & Pages from 2015-2019

Create a list of urls that represent the years

In [4]:
movie_years = ['2019','2018','2017','2016','2015']
def movie_years_url (years):
    movie_url = []
    for year in years:
        movie_url.append(moviedb_url.replace('2019',year))
    return movie_url
moviedb_list_years = movie_years_url(movie_years)

Create a list to iterate over with the page numbers

In [47]:
def pagenumber_list (pb,pe):
    numberlist = []
    if pb == pe:
        return pb
    else:
        while pb < pe+1:
            numberlist.append(pb)
            pb += 1
    return numberlist

moviedb_pages = pagenumber_list(1,100)

Create a List of urls that has the page numbers and the years properly iterated over

In [48]:
def movie_page_url (movielist):
    result = []
    count = 0
    for url in movielist:
        for page in moviedb_pages:
            result.append(url.replace('page=1',f'page={moviedb_pages[page-1]}'))
    return result       

moviedb_pages_url = movie_page_url(moviedb_list_years)

500

This comes out to a total of 500 URLs to run the API through with 20 results per URL making the total raw data at 10,000 rows worth of data.

## Run the API over the list of finalized urls to get all the data from 2015-2019

Running the final formula to run the requests and create a dataframe with 10,000 Rows of data

In [49]:
def moviedb_df (url_list):
    moviedb_dataframe = pd.DataFrame()
    for url in url_list:
        moviedb_dataframe = pd.concat([moviedb_dataframe,(pd.DataFrame(requests.get(url).json()['results']))],sort=True)
    return moviedb_dataframe

moviedb_final = moviedb_df(moviedb_pages_url)

Getting more detailed movie info by running another API

In [ ]:
def movieinfo_df (movie_id):
    movieinfo_list = []
    for ID in movie_id:
        movieinfo = requests.get(f'https://api.themoviedb.org/3/movie/{ID}?api_key={moviedb_key}&language=en-US').json()
        movieinfo_list.append(movieinfo)
    return movieinfo_list

movieinfo = movieinfo_df(moviedb_final.id)
moviedb_info_df = pd.DataFrame(movieinfo)

Combining the two dataframes with the data that is useful for the final dataframe

In [376]:
moviedb_final_filtered_df = moviedb_final[['id','genre_ids','original_title','title','release_date','popularity','vote_average','vote_count']]
moviedb_info_filtered_df = moviedb_info_df[['id','budget','revenue','imdb_id']]
MovieDB_df = pd.merge(moviedb_final_filtered_df, moviedb_info_filtered_df, on='id')

## Now to clean the dataframe to make it presentable

We will start by removing all duplicate values

In [377]:
MovieDB_df = MovieDB_df.drop_duplicates(subset = 'id')

Since the Genres are still IDs so we will convert them into strings with the genres in readable format

In [378]:
moviedb_genres = requests.get(f'https://api.themoviedb.org/3/genre/movie/list?api_key={moviedb_key}&language=en-US')
movie_genre = pd.DataFrame(moviedb_genres.json()['genres'])
movie_genre = movie_genre.set_index('id')
genre = movie_genre['name'].to_dict()

Simple Formula to replace the values

In [379]:
def genre_repl(list_):
    li =[]
    for i in list_:
        if i in genre.keys():
            li.append(genre[i])
    return li

Replacing the old column with the new column with the proper values

In [380]:
MovieDB_df['genre_ids'] = MovieDB_df['genre_ids'].apply(lambda x : genre_repl(x))

Now to clean up the revenue and budget columns so it represents actual currency and update the averages to represent precentages

In [382]:
def currency_convert(number):

    return ('${:,.2f}'.format(number))

In [383]:
MovieDB_df['revenue'] = MovieDB_df['revenue'].apply(lambda x: currency_convert(x))

In [384]:
MovieDB_df['budget'] = MovieDB_df['budget'].apply(lambda x: currency_convert(x))

In [387]:
MovieDB_df['vote_average'] = MovieDB_df['vote_average']*10

## Finalize the dataframe and then convert it into a csv

Take a quick last look at the dataframe to make sure everything looks good with the values.

In [388]:
MovieDB_df

,id,genre_ids,original_title,title,release_date,popularity,vote_average,vote_count,budget,revenue,imdb_id
0,475303,"[Comedy, Romance]",A Rainy Day in New York,A Rainy Day in New York,2019-07-26,1305.559,67.0,591,$0.00,$0.00,tt7139936
1,512200,"[Action, Adventure, Comedy, Fantasy]",Jumanji: The Next Level,Jumanji: The Next Level,2019-12-04,272.710,68.0,1901,"$125,000,000.00","$310,830,000.00",tt7975244
2,419704,"[Adventure, Drama, Mystery, Science Fiction, T...",Ad Astra,Ad Astra,2019-09-17,271.489,60.0,2483,"$87,500,000.00","$127,175,922.00",tt2935510
3,496243,"[Comedy, Drama, Thriller]",기생충,Parasite,2019-05-30,148.987,86.0,4756,"$11,363,000.00","$201,055,038.00",tt6751668
4,330457,"[Adventure, Animation, Family]",Frozen II,Frozen II,2019-11-20,124.257,71.0,2614,"$33,000,000.00","$1,330,764,959.00",tt4520988
...,...,...,...,...,...,...,...,...,...,...,...
17292,325071,"[TV Movie, Mystery]",Garage Sale Mystery: The Deadly Room,Garage Sale Mystery: The Deadly Room,2015-04-11,3.233,65.0,30,$0.00,$0.00,tt4460190
17293,18044,"[Horror, Science Fiction]",Ogre,Ogre,2008-03-08,3.237,40.0,22,$0.00,$0.00,tt0923824
17294,148656,"[Fantasy, Family, TV Movie]",The Good Witch's Charm,The Good Witch's Charm,2012-10-27,4.618,72.0,43,$0.00,$0.00,tt2273004
17295,324300,[Documentary],Rolling Papers,Rolling Papers,2015-03-15,3.102,60.0,9,$0.00,$0.00,tt3621360


Now convert the dataframe into a csv

In [389]:
MovieDB_df.to_csv('/Users/takehiroyasuoka/Documents/Flatiron/Flatiron Repos/Flatiron_Mod_1_Project/MovieDB_df.csv')